In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from functools import reduce
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
driver = webdriver.Firefox(executable_path='C:\Program Files (x86)\geckodriver.exe')
driver.get("https://geographic.org/streetview/usa/ga/index.html")

In [3]:
def page_loaded():
    main = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

In [4]:
def hit_escape():
    try:
        page_loaded()
        driver.find_element_by_tag_name('body').send_keys(Keys.ESCAPE)
    except:
        driver.quit()

In [5]:
def load_delay():
    try:
        page_loaded()
        
    except:
        driver.quit()
        
    time.sleep(1)

In [6]:
def back_arrow():
    try:
        page_loaded()
        driver.back()
        hit_escape()
    except:
        pass
        #driver.quit()

In [7]:
def click_a_link(current_link):
    try:
        page_loaded()
        current_link.click()
        hit_escape()
    except:
        driver.quit()

In [8]:
def crawl_town(link):
    
    click_a_link(link)
    
    load_delay()
    local_streets = [(element.text, county) for element in driver.find_elements_by_xpath
                         ("//li/a[contains(@href,'place=')]")]
    back_arrow()
    return (local_streets)

In [9]:
def crawl_county(link):
    
    click_a_link(link)
    
    load_delay()
    no_of_towns = len(driver.find_elements_by_xpath("//li/a[contains(@href,'.html')]"))
    
    town_streets = reduce(lambda a,b: a + crawl_town(driver.find_elements_by_xpath("//li/a[contains(@href,'.html')]")[b]),
                          range(no_of_towns), [])
    
    
    back_arrow()
    return town_streets       

In [10]:
county_links = driver.find_elements_by_xpath("//li/a[contains(@href,'index.html')]")
number_of_counties = len(county_links)

In [11]:
streets = []
for i in range(number_of_counties):
    county_link = driver.find_elements_by_xpath("//li/a[contains(@href,'index.html')]")[i]
    county = county_link.text
    streets = streets + crawl_county(county_link)

In [12]:
pd.DataFrame(streets, columns = ['street', 'county']).to_csv('Georgia streets.csv')

In [13]:
pd.DataFrame(streets, columns = ['street', 'county']).head().style.hide_index()

street,county
1st Street,Appling
2nd Street,Appling
A P Hunter Road,Appling
Acorn Lane,Appling
Addie Lane,Appling


In [14]:
pd.DataFrame(streets, columns = ['street', 'county']).tail()

,street,county
2365,Water Street Circle,Baker
2366,"Water Street, North",Baker
2367,William Irvin Road,Baker
2368,Williams Road,Baker
2369,Willow Nook Road,Baker
